# Authorization Code Flow Example

This is an interactive, redirection-based flow. The client must be capable of interacting with the resource owner's user-agent (typically a web browser) and capable of receiving incoming requests (via redirection) from the  authorization server.

In [ ]:
#!pwsh

$PSVersionTable.PSVersion

The client application must call the waters_connect authorization endpoint.
In this interactive step, the user is required to authenticate by entering his waters_connect credentials:

In [ ]:
#!pwsh

$ie = new-object -com "InternetExplorer.Application"
$ie.navigate("https://v2.unifiapi.com:48333/connect/authorize?client_id=jupyter&scope=webapi&response_type=code&redirect_uri=http://localhost:8888")
$ie.visible = $true

In case of successful authentication, the waters_connect Identity Provider redirects the browser back to the user application.
Note that the redirect URI is explicitly specified as query string on the call to the authorization endpoint above.

Next, let's examine the full redirect URI:

In [ ]:
$url = [uri]$ie.LocationURL
$decoded = [uri]::UnescapeDataString($url)
$u = [System.Web.HttpUtility]::UrlDecode($url)
$u

The URI contains the authorization code issued by the waters_connect Identity Provider.
The client application must grab the authorization code and exchange it for an access token in order to be able to access waters_connect protected resources.

In [ ]:
$url_parts_regex = '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?'

if ($url -match $url_parts_regex) {
    $url_parts = @{
        'Scheme' = $Matches[2];
        'Server' = $Matches[4];
        'Path' = $Matches[5];
        'QueryString' = $Matches[7];
        'QueryStringParts' = @{}
    }

    foreach ($qs in $url_parts.QueryString.Split('&')) {
        $qs_key, $qs_value = $qs.Split('=')
        $url_parts.QueryStringParts.Add(
            [uri]::UnescapeDataString($qs_key),
            [uri]::UnescapeDataString($qs_value)
        ) | Out-Null
    }
} else {
    Throw [System.Management.Automation.ParameterBindingException] "Invalid URL Supplied"
}

$url_parts.QueryStringParts

In order to exchange the authorization code with an access token, the client application must do a POST request to the waters_connect `token` endpoint, as follows:

In [ ]:
#!pwsh
$out = curl.exe -X POST --insecure -H "Content-Type: application/x-www-form-urlencoded" -d "client_id=jupyter&client_secret=secret&grant_type=authorization_code&code=ef0d2f603ce8a6988bc5ceb598e45396c6a26e00e6652b38200481a1ee24e9b9&redirect_uri=http://localhost:8888" https://v2.unifiapi.com:48333/connect/token
$out

In [ ]:
$access_token = "eyJhbGciOiJSUzI1NiIsImtpZCI6IjU4MjgxNjNENzhFM0VFOEY0MjY3OTgwOEQ1MTZDQjA3NkM4QTY0MkEiLCJ0eXAiOiJKV1QiLCJ4NXQiOiJXQ2dXUFhqajdvOUNaNWdJMVJiTEIyeUtaQ28ifQ.eyJuYmYiOjE2NDYwMzc1MjIsImV4cCI6MTY0NjA0MTEyMiwiaXNzIjoiaHR0cHM6Ly9lYzJhbWF6LWV1dmQ4c2s6NDgzMzMvIiwiYXVkIjpbImh0dHBzOi8vZWMyYW1hei1ldXZkOHNrOjQ4MzMzL3Jlc291cmNlcyIsIndlYmFwaSJdLCJjbGllbnRfaWQiOiJqdXB5dGVyIiwic3ViIjoiNDUwMWEwMDYtMWQyZC00Y2UzLWFhYjYtMzYwYTgxMDQ4NTkzIiwiYXV0aF90aW1lIjoxNjQ2MDM3NDg4LCJpZHAiOiJsb2NhbCIsIm5hbWUiOiJhZG1pbmlzdHJhdG9yIiwiZ2l2ZW5fbmFtZSI6IndhdGVyc19jb25uZWN0IiwiZmFtaWx5X25hbWUiOiJBZG1pbmlzdHJhdG9yIiwicm9sZSI6IjdiYjkwZDUzLTY5ZTktNDNkMS1hZjFmLTBhYmZkMzAwYThjNyIsImNsaWVudE1hY2hpbmVOYW1lIjoiSVAtMTcyLTMxLTIxLTYiLCJzY29wZSI6WyJ3ZWJhcGkiXSwiYW1yIjpbInB3ZCJdfQ.mFSFTXJZpNKna8JSooaHESB0Ep2Cb2w6GIOqI2WdWuCHIqFhfwyo8f9ux4WZZt-VM3uhp8vCBA6KoTNixNxPZbm0wuJAQTa4Jd5S16BZOJVF-TsPD28I2McM-zymhh8PAollhY2LtiJm7dV81wq6oQ4OmkYxrtZp5kTj6f7HqbRyaqso_TKDQupDL-iVYUMwKJO3I6QHQTJgMxrMmLz44JvS3BnKrtZISXsRijnRA-5o7AIZtomubPtcJsMPqLcbn8Kr964cO9IxQfx91py0wLmVchjcIqVfcghMq6z4ZILpzGzuOpfqHNjNvYO6O5j5EhmPeO97_xUikPQpCcxSdHPWESvJKY36uVCH19HOBXlAhukQA7Tgssxo1nSwkyC29B030lrYnZ9OVitTnSeEBS3mzFBb5F8svMe_BcSkDEFyzkQFqig8wbCO2kn6Z3SVGpWK8ngtfmq6lNUqlCXqPpzua0-uBs4bxexi5-WeKMuHZjGCI9KS4OnYypt7ZhJtuS3v22sOoZ3_pE-aqnVGMHf3uwCoqPpiLu0Xqw9f50Tor-Zm2jLVWWqzBl_gf3KaQj3zlyZs1nihJkEDVl5rlOD9ef6RZSvaE_3QuuWRn4HAoUsr-hXhPTyKNdkXD-39qaQD2eC_ch1e4B3X3guAqRR3O2iknMZlIn7GopFb0dw"

The access token has to be put on the `Authorization` header when calling waters_connect protected endpoints:

In [ ]:
curl.exe -X GET --insecure -H "Authorization: Bearer $access_token" "https://v2.unifiapi.com:48444/waters_connect/v1.0/folders"